In [15]:
import csv
import requests
import pandas as pd
from tqdm import tqdm
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [16]:
def read_reports_from_csv():
    df = pd.read_csv("All_Scraped.csv")
    report_list = df["Report"].tolist()
    return report_list

previous_reports = read_reports_from_csv()

In [17]:
options = Options()
# options.add_argument('--headless=new')
# options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(options=options)
driver.maximize_window()

url = 'https://www.airdropbob.com/'

driver.get(url)
element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.ID, 'resultsRow')))

In [18]:
def get_data(projects, new_scraped_writer, all_scraped_writer):
    try:
        projects.pop(4)
        try:
            projects.pop(9)
        except:
            pass
    except:
        pass
    projects = [project.get_attribute('href') for project in projects]

    for project in tqdm(projects, desc='Projects Scraping'):
        if project not in previous_reports:
            data = {}

            response = requests.get(project)
            soup = BeautifulSoup(response.content, 'lxml')

            try:
                data['Name'] = soup.select_one('div.provider-name').text
            except:
                return None
            try:
                data['Website'] = soup.select_one('a.btn.btn-secondary.btn-block').get('href')
            except:
                data['Website'] = ''
            data['Report'] = project
            data['Telegram'] = ''
            data['Twitter'] = ''
            data['Discord'] = ''

            new_scraped_writer.writerow(data)
            all_scraped_writer.writerow(data)
        else:
            continue

In [19]:
header = ['Name','Website','Telegram','Twitter','Discord','Report']
with open("All_Scraped.csv", mode='a', newline='', encoding='utf-8') as all_scraped_file, open("New_Scraped.csv", mode='a', newline='', encoding='utf-8') as new_scraped_file:
    new_scraped_writer = csv.DictWriter(new_scraped_file, fieldnames=header)
    all_scraped_writer = csv.DictWriter(all_scraped_file, fieldnames=header)

    next_button = driver.find_elements(By.CSS_SELECTOR, 'a.page-link')[-1]
    while 'disabled' not in next_button.get_attribute('class'):
        projects = driver.find_elements(By.CSS_SELECTOR, 'div.col-md-4 > a')
        get_data(projects, new_scraped_writer, all_scraped_writer)

        driver.execute_script("window.scrollBy(0, 1200);")
        sleep(1)

        next_button = driver.find_elements(By.CSS_SELECTOR, 'a.page-link')[-1]
        next_button.click()
        sleep(5)

        next_button = driver.find_elements(By.CSS_SELECTOR, 'a.page-link')[-1]
    
    projects = driver.find_elements(By.CSS_SELECTOR, 'div.col-md-4 > a')
    get_data(projects[:10], new_scraped_writer, all_scraped_writer)

Projects Scraping: 100%|██████████| 2/2 [00:00<00:00, 2948.54it/s]


In [ ]:
header = ['Name','Website','Telegram','Twitter','Discord','Report']
with open("All_Scraped.csv", mode='a', newline='', encoding='utf-8') as all_scraped_file, open("New_Scraped.csv", mode='a', newline='', encoding='utf-8') as new_scraped_file:
    new_scraped_writer = csv.DictWriter(new_scraped_file, fieldnames=header)
    all_scraped_writer = csv.DictWriter(all_scraped_file, fieldnames=header)
    #     print('Writing to file...')
    get_data(projects[:10], new_scraped_writer, all_scraped_writer)

In [ ]:
driver.execute_script("window.scrollBy(0, 1300);")

In [ ]:
r = requests.get('https://www.airdropbob.com/jok-ai-labs/airdrop-jok-ai-labs')
soup = BeautifulSoup(r.content, 'lxml')

In [ ]:
soup.select_one('a.btn.btn-secondary.btn-block').get('href')